In [1]:
import sys
sys.path.insert(0, "../scripts")
import vavg_indoor_air as v
import wall_cond as w

from sympy import * 
from spb import *

import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly as plotly

In [2]:
v1 = v.VavgIndoorAir(v.PhysicalConstants())

In [10]:
times, temps = v1.calc_transient_indoor_air()

In [12]:
n_colors = len([ix_time for ix_time,_ in enumerate(v1.all_wall_temps) if ix_time%(10*4)==0]) + 1

colors = px.colors.sample_colorscale("aggrnyl", [n/(n_colors -1) for n in range(n_colors)])

len(colors)

7

In [19]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, subplot_titles=("Wall Temps", "Room Temp"))



cix=0
for ix_time, x_at_time in enumerate(v1.all_wall_temps):
    if ix_time%(10*4)== 0:
        fig.add_trace(go.Scatter(x=v1.twall.x_vals, y=x_at_time, mode='lines+markers', name=f"t={times[ix_time]}", line=dict(color=colors[cix])), row = 1, col = 1)


        fig.add_trace(go.Scatter(x=[times[ix_time]], y=[temps[ix_time]], mode='lines+markers', name=f"t={times[ix_time]}", showlegend=False, line=dict(color=colors[cix])), row = 1, col = 2)
        cix+=1

fig['layout']['yaxis']['title']='Temperature (K)'
fig['layout']['xaxis']['title']='Wall Nodes Distance'
fig['layout']['xaxis2']['title']='Time (s)'
fig.show()

In [15]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, subplot_titles=("Wall Temps", "Room Temp"))



cix=0
for ix_time, x_at_time in enumerate(v1.all_wall_temps):
    if ix_time%(10*4)== 0:
        fig.add_trace(go.Scatter(x=v1.twall.x_vals, y=x_at_time, mode='lines+markers', name=f"t={times[ix_time]}", line=dict(color=colors[cix])), row = 1, col = 1)

        # print(temps[ix_time])

        fig.add_trace(go.Scatter(x=[0], y=[temps[ix_time]], mode='lines+markers', name=f"t={times[ix_time]}", showlegend=False, line=dict(color=colors[cix])), row = 1, col = 2)
        cix+=1
fig.show()

In [4]:
t = w.TransientWallConduction(w.FabricPhysicalConstants(),v1.times, v1.pc.T0)
t


In [5]:
t.get_ext_temps()

2022-07-27 09:00:00    290.261111
2022-07-27 09:00:15    290.261111
2022-07-27 09:00:30    290.261111
2022-07-27 09:00:45    290.261111
2022-07-27 09:01:00    290.261111
                          ...    
2022-07-27 09:58:45    290.261111
2022-07-27 09:59:00    290.261111
2022-07-27 09:59:15    290.261111
2022-07-27 09:59:30    290.261111
2022-07-27 09:59:45    290.261111
Freq: 15S, Name: Ambient Temp, Length: 240, dtype: float64

In [6]:
t.calc_Tx_at_t(0, 292)

array([290.26185543, 290.58209877, 290.90308642, 291.22407407,
       291.54506173, 291.86604938, 292.18703704, 292.50802469,
       292.82901235, 293.14912235])

In [7]:
t.calc_Tx_at_t(2, 293)

array([0.03365346, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.03397101])

In [8]:
t.T_inits

array([290.26111111, 290.58209877, 290.90308642, 291.22407407,
       291.54506173, 291.86604938, 292.18703704, 292.50802469,
       292.82901235, 293.15      ])

In [9]:
t.Ttx

array([[290.26111111, 290.58209877, 290.90308642, ..., 292.50802469,
        292.82901235, 293.15      ],
       [290.26185543, 290.58209877, 290.90308642, ..., 292.50802469,
        292.82901235, 293.14912235],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])